In [10]:
from pyvisionproductsearch import ProductSearch, ProductCategories
from google.cloud import storage
from google.cloud import firestore
import pandas as pd
from google.cloud import vision
from google.cloud.vision import types
from utils import detectLabels, detectObjects
import io
from tqdm.notebook import tqdm
import os
from dotenv import load_dotenv
load_dotenv()

True

In [81]:
# Fill these out with your own values
# GCP config
GCP_PROJECTID="YOUR_PROJECT_ID"
BUCKET="YOUR_BUCKET"
CREDS="key.json"
PRODUCT_SET="YOUR_PRODUCT_SET"
INSPO_BUCKET = "YOUR_INSPO_PIC_BUCKET"
# If your inspiration pictures are in a subfolder, list it here:
INSPO_SUBFOLDER = "YOUR_SUBFOLDER_NAME"

In [12]:
# To use this notebook, make a copy of .env_template --> .env and fill out the fields!
ps = ProductSearch(GCP_PROJECTID, CREDS, BUCKET)
productSet = ps.getProductSet(PRODUCT_SET)

### Download fashion influence pics and filter them by "Fashion" images

In [14]:
# For each fashion inspiration pic, check to make sure that it's 
# a "fashion" picture. Ignore all other pics
storage_client = storage.Client()
blobs = list(storage_client.list_blobs(INSPO_BUCKET, prefix=INSPO_SUBFOLDER))
uris = [os.path.join("gs://", blobs[0].bucket.name, x.name)
        for x in blobs if '.jpg' in x.name]
urls = [x.public_url for x in blobs if '.jpg' in x.name]

fashionPics = []
for uri, url in tqdm(list(zip(uris, urls))):
    labels = detectLabels(image_uri=uri)
    if any([x.description == "Fashion" for x in labels]):
        fashionPics.append((uri, url))
fashion_pics = pd.DataFrame(fashionPics, columns=["uri", "url"])

In [ ]:
# Run this line to verify you can actually search your product set using a picture
productSet.search("apparel", image_uri=fashion_pics['uri'].iloc[0])

Example Response:

    {'score': 0.7648860812187195,
      'label': 'Shoe',
      'matches': [{'product': <pyvisionproductsearch.ProductSearch.ProductSearch.Product at 0x14992d2e0>,
        'score': 0.35719582438468933,
        'image': 'projects/yourprojectid/locations/us-west1/products/high_rise_white_jeans_pants/referenceImages/6550f579-6b26-433a-8fa6-56e5bbca95c1'},
       {'product': <pyvisionproductsearch.ProductSearch.ProductSearch.Product at 0x14992d5b0>,
        'score': 0.32596680521965027,
        'image': 'projects/yourprojectid/locations/us-west1/products/white_boot_shoe/referenceImages/56248bb2-9d5e-4004-b397-6c3b2fb0edc3'},
       {'product': <pyvisionproductsearch.ProductSearch.ProductSearch.Product at 0x14a423850>,
        'score': 0.26240724325180054,
        'image': 'projects/yourprojectid/locations/us-west1/products/tan_strap_sandal_shoe/referenceImages/f970af65-c51e-42e8-873c-d18080f00430'}],
      'boundingBox': [x: 0.6475263833999634
      y: 0.8726409077644348
      , x: 0.7815263271331787
      y: 0.8726409077644348
      , x: 0.7815263271331787
      y: 0.9934644103050232
      , x: 0.6475263833999634
      y: 0.9934644103050232
      ]},
     {'score': 0.8066604733467102,
      'label': 'Shorts',
      'matches': [{'product': <pyvisionproductsearch.ProductSearch.ProductSearch.Product at 0x106a4fa60>,
        'score': 0.27552375197410583,
        'image': 'projects/yourprojectid/locations/us-west1/products/white_sneaker_shoe_*/referenceImages/a109b530-56ff-42bc-ac73-d60578b7f363'},
       {'product': <pyvisionproductsearch.ProductSearch.ProductSearch.Product at 0x106a4f400>,
        'score': 0.2667400538921356,
        'image': 'projects/yourprojectid/locations/us-west1/products/grey_vneck_tee_top_*/referenceImages/cc6f873c-328e-481a-86fb-a2116614ce80'},
       {'product': <pyvisionproductsearch.ProductSearch.ProductSearch.Product at 0x106a4f8e0>,
        'score': 0.2606571912765503,
        'image': 'projects/yourprojectid/locations/us-west1/products/high_rise_white_jeans_pants_*/referenceImages/360b26d8-a844-4a83-bf97-ef80f2243fdb'},
       {'product': <pyvisionproductsearch.ProductSearch.ProductSearch.Product at 0x106a4fb80>],
      'boundingBox': [x: 0.4181176424026489
      y: 0.40305882692337036
      , x: 0.6837647557258606
      y: 0.40305882692337036
      , x: 0.6837647557258606
      y: 0.64000004529953
      , x: 0.4181176424026489
      y: 0.64000004529953
      ]}]

The response above returns a set of matches for each item identified in your inspiration photo.

In the example above, "Shorts" and "Shoes" were recognized. For each of those items, a bounding box is returned that indicates where the item is in the picture.

For each matched item in your closet, a `Product` object is returned along with its image id and a confidence score.

### Get clothing matches

We want to make sure that when we recommend users similar items that we respect clothing type. 

For example, the Product Search API might (accidentally) return a dress as a match for a shirt, but we wouldn't want to expose that to the end user. So this function--getBestMatch--sorts through the results returned by the API and makes sure that a. only the highest confidence match for each item is returned and b. that the item types match.

In [77]:
# The API sometimes uses different names for similar items, so this
# function tells you whether two labels are roughly equivalent
def isTypeMatch(label1, label2):
    # everything in a single match group are more or less synonymous
    matchGroups = [("skirt", "miniskirt"), 
               ("jeans", "pants"), 
               ("shorts"),
               ("jacket", "vest", "outerwear", "coat", "suit"),
               ("top", "shirt"),
               ("dress"),
               ("swimwear", "underpants"),
               ("footwear", "sandal", "boot", "high heels"),
               ("handbag", "suitcase", "satchel", "backpack", "briefcase"),
               ("sunglasses", "glasses"),
               ("bracelet"),
               ("scarf", "bowtie", "tie"),
               ("earrings"),
               ("necklace"),
               ("sock"),
               ("hat", "cowboy hat", "straw hat", "fedora", "sun hat", "sombrero")]
    for group in matchGroups:
        if label1.lower() in group and label2.lower() in group:
            return True
    return False

In [79]:
def getBestMatch(searchResponse):
    label = searchResponse['label']
    matches = searchResponse['matches']
    viableMatches = [match for match in matches if any([isTypeMatch(label, match['product'].labels['type'])])]
    return max(viableMatches, key= lambda x: x['score']) if len(viableMatches) else None


After we run `getBestMatch` above, we're left with a bunch of items from our own closet that match our inspiration picture. But the next step is transform those matches into an "outfit," and outfits have rules: you can't wear a dress and pants at the same time (probably). You usually only wear one type of shoe. This next function, `canAddItem`, allows us to add clothing items to an outfit one at a time without breaking any of the "rules" of fashion.

In [22]:
def canAddItem(existingArray, newType):
    bottoms = {"pants", "skirt", "shorts", "dress"}
    newType = newType.lower()
    # Don't add the same item type twice
    if newType in existingArray:
        return False
    if newType == "shoe":
        return True
    # Only add one type of bottom (pants, skirt, etc)
    if newType in bottoms and len(bottoms.intersection(existingArray)):
        return False
    # You can't wear both a top and a dress
    if newType == "top" and "dress" in existingArray:
        return False
    return True

Finally, we need a function that allows us to evaluate how "good" an outfit recommendation is. We'll do this by creating a score function. This part is creative, and you can do it however you like. Here are some example score functions:

In [73]:
# Option 1: sum up the confidence scores for each closet item matched to the inspo photo
def scoreOutfit1(matches):
    if not matches:
        return 0
    return sum([match['score'] for match in matches]) / len(matches)

# Option 2: Sum up the confidence scores only of items that matched with the inspo photo 
# with confidence > 0.3. Also, because shoes will match most images _twice_ 
# (because people have two feet), only count the shoe confidence score once
def scoreOutfit2(matches):
    if not len(matches):
        return 0
    
    noShoeSum = sum([x['score'] for x in matches if (x['score'] > 0.3 and not isTypeMatch("shoe", x["label"]))])
    shoeScore = 0
    try:
        shoeScore = max([x['score'] for x in matches if isTypeMatch("shoe", x["label"])])
    except:
        pass
    return noShoeSum + shoeScore * 0.5 # half the weight for shoes

Great--now that we have all our helper functions written, let's combine them into one big function for 
constructing an outfit and computing its score!

In [92]:
def getOutfit(imgUri, verbose=False):
    # 1. Search for matching items
    response = productSet.search("apparel", image_uri=imgUri)
    if verbose:
        print("Found matching " + ", ".join([x['label'] for x in response]) + " in closet.")

    clothes = []
    # 2. For each item in the inspo pic, find the best match in our closet and add it to 
    # the outfit array
    for item in response:
        bestMatch = getBestMatch(item)
        if not bestMatch:
            if verbose:
                print(f"No good match found for {item['label']}")
            continue
        if verbose:
            print(f"Best match for {item['label']} was {bestMatch['product'].displayName}")
        clothes.append(bestMatch)

    # 3. Sort the items by highest confidence score first
    clothes.sort(key=lambda x: x['score'], reverse=True)

    # 4. Add as many items as possible to the outfit while still
    # maintaining a logical outfit
    outfit = []
    addedTypes = []
    for item in clothes:
        itemType = item['product'].labels['type'] # i.e. shorts, top, etc
        if canAddItem(addedTypes, itemType):
            addedTypes.append(itemType)
            outfit.append(item)
            if verbose:
                print(f"Added a {itemType} to the outfit")

    # 5. Now that we have a whole outfit, compute its score!
    score1 =  scoreOutfit1(outfit)
    score2 = scoreOutfit2(outfit)
    if verbose:
        print("Algorithm 1 score: %0.3f" % score1)
        print("Algorithm 2 score: %0.3f" % score2)
    return (outfit, score1, score2)
    

In [ ]:
getOutfit(fashion_pics.iloc[0]['uri'], verbose=True)

Output:

        Found matching Shorts, Shoe in closet.
        Best match for Shorts was high_rise_white_shorts_*
        No good match found for Shoe
        Added a shorts to the outfit
        Algorithm 1 score: 0.247
        Algorithm 2 score: 0.000
        {'outfit': [{'product': <pyvisionproductsearch.ProductSearch.ProductSearch.Product at 0x149fa6760>,
           'score': 0.24715223908424377,
           'image': 'projects/yourprojectid/locations/us-west1/products/high_rise_white_shorts_*/referenceImages/71cc9936-2a35-4a81-8f43-75e1bf50fc22'}],
         'score1': 0.24715223908424377,
         'score2': 0.0}

### Add Data to Firestore

Now that we have a way of constructing and scoring outfits, let's add them to Firestore
so we can later use them in our app.

In [88]:
db = firestore.Client()
userid = u"youruserd" # I like to store all data in Firestore as users, incase I decide to add more in the future!
thisUser = db.collection(u'users').document(userid)
outfits = thisUser.collection(u'outfitsDEMO')

In [91]:
# Go through all of the inspo pics and compute matches.
for row in fashion_pics.iterrows():
    srcUrl = row[1]['url']
    srcUri = row[1]['uri']
    (outfit, score1, score2) = getOutfit(srcUri, verbose=False)
    
    # Construct a name for the source image--a key we can use to store it in the database
    srcId = srcUri[len("gs://"):].replace("/","-")
    
    # Firestore writes json to the database, so let's construct an object and fill it with data
    fsMatch = {
        "srcUrl": srcUrl,
        "srcUri": srcUri,
        "score1": score1,
        "score2": score2,
    }
    # Go through all of the outfit matches and put them into json that can be
    # written to firestore
    theseMatches = []
    for match in outfit:
        image = match['image']
        imgName = match['image'].split('/')[-1]
        name = match['image'].split('/')[-3]
        # The storage api makes these images publicly accessible through url
        imageUrl = f"https://storage.googleapis.com/{BUCKET}/" + imgName
        label = match['product'].labels['type']
        score = match['score']

        theseMatches.append({
            "score": score,
            "image": image,
            "imageUrl": imageUrl,
            "label": label
        })
    fsMatch["matches"] = theseMatches
    # Add the outfit to firestore!
    outfits.document(srcId).set(fsMatch)

Added a shorts to the outfit


Voila! Now you have a bunch of matches to recommend in Firestore! Just build a nice frontend to back it up!